## 1. Import Required Libraries

Import all necessary Python libraries for Azure services, data processing, and the custom migration modules.

In [46]:
import os
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict, Any
import dotenv
import importlib

# Azure SDK imports
from azure.data.tables import TableServiceClient
from azure.storage.blob import BlobServiceClient

# Custom modules (make sure backend is in path)
import sys
sys.path.append('.')
from backend.BlobStorage import ConversationBlobStore, ConversationMetadataStore

# Load environment variables
dotenv.load_dotenv(override=True)

print("✅ All libraries imported successfully")
print(f"📅 Migration started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ All libraries imported successfully
📅 Migration started at: 2025-07-04 15:52:48


## 2. Set Up Azure Table and Blob Storage Clients

Configure Azure Storage clients using environment variables. This connects to your existing storage account.

In [47]:
importlib.reload(sys.modules['backend.BlobStorage'])
# Build Azure Storage connection string
connection_string = f"AccountName={os.getenv('AZURE_STORAGE_ACCOUNT_NAME')};AccountKey={os.getenv('AZURE_STORAGE_ACCOUNT_KEY')};EndpointSuffix=core.windows.net"

# Initialize Table Storage client
table_client = TableServiceClient.from_connection_string(conn_str=connection_string)
old_chatlogs_table = table_client.get_table_client(table_name="chatlogs")
chatlogs_table = table_client.get_table_client(table_name=os.getenv("CONVERSATION_METADATA_TABLE_NAME"))

# Initialize Blob Storage client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Initialize our custom migration classes
blob_store = ConversationBlobStore(connection_string, container_name=os.getenv("CONVERSATION_CONTAINER_NAME"))
metadata_store = ConversationMetadataStore(chatlogs_table, blob_store)

print("✅ Azure clients initialized successfully")
print(f"📦 Storage account: {os.getenv('AZURE_STORAGE_ACCOUNT_NAME')}")
print(f"🗂️ Table: {os.getenv('CONVERSATION_METADATA_TABLE_NAME')}")
print(f"📁 Container: {os.getenv('CONVERSATION_CONTAINER_NAME')}")

✅ Azure clients initialized successfully
📦 Storage account: taicdocumentsearcherdata
🗂️ Table: smartassistantconversationsmetadata
📁 Container: smartassistantconversations


## 3. Define Migration Functions

Create helper functions to inspect and migrate conversation data safely.

In [ ]:
def inspect_conversation_entity(entity: Dict) -> Dict:
    """
    Inspect a Table Storage conversation entity to understand its structure.
    """
    info = {
        "username": entity.get("PartitionKey", "unknown"),
        "conversation_id": entity.get("RowKey", "unknown"),
        "title": entity.get("conversation_title", "Untitled"),
        "last_updated": entity.get("last_updated", "unknown"),
       # No storage_type column anymore
        "message_chunks": []
    }
    
    # Find message chunks
    message_keys = [key for key in entity.keys() if key.startswith("messages")]
    info["chunk_count"] = len(message_keys)
    
    # Calculate total size
    total_size = 0
    for key in sorted(message_keys):
        chunk_size = len(entity[key]) if entity[key] else 0
        total_size += chunk_size
        info["message_chunks"].append({
            "key": key,
            "size": chunk_size
        })
    
    info["total_size"] = total_size
    return info

def reconstruct_conversation_json(entity: Dict) -> tuple[list[Dict], str]:
    """
    Reconstruct conversation JSON from chunked table storage entity.
    Returns (messages, error_message)
    """
    try:
        # Get all message chunks
        message_keys = [key for key in entity.keys() if key.startswith("messages")]
        if not message_keys:
            return None, "No message chunks found"
        
        # Reconstruct complete JSON string
        all_messages = [entity[key] for key in sorted(message_keys)]
        complete_message = "".join(all_messages)
        
        # Parse JSON
        messages = json.loads(complete_message)
        return messages, None
        
    except json.JSONDecodeError as e:
        return None, f"JSON decode error: {str(e)}"
    except Exception as e:
        return None, f"Reconstruction error: {str(e)}"

def migrate_single_conversation(entity: Dict) -> Dict:
    """
    Migrate a single conversation from Table Storage to Blob Storage.
    Returns migration result with status and details.
    """
    result = {
        "username": entity.get("PartitionKey"),
        "conversation_id": entity.get("RowKey"),
        "status": "pending",
        "error": None,
        "blob_name": None,
        "message_count": 0
    }
    
    try:
        # Check if blob already exists (new logic)
        blob_name = f"{result['username']}/{result['conversation_id']}.json"
        if blob_store.retrieve_conversation_blob(result["username"], result["conversation_id"]):
            result["status"] = "already_migrated"
            result["blob_name"] = blob_name
            return result
        
        # Reconstruct conversation
        messages, error = reconstruct_conversation_json(entity)
        if error:
            result["status"] = "failed"
            result["error"] = error
            return result
        
        # Migrate to blob storage
        success = metadata_store.create_or_update_conversation(
            username=result["username"],
            conversation_id=result["conversation_id"],
            history=messages,
            conversation_title=entity.get("conversation_title", "Migrated Conversation")
        )
        
        if success:
            result["status"] = "success"
            result["message_count"] = len(messages)
            result["blob_name"] = blob_name
        else:
            result["status"] = "failed"
            result["error"] = "Failed to store in blob storage"
            
    except Exception as e:
        result["status"] = "failed"
        result["error"] = str(e)
    
    return result

print("✅ Migration functions updated to use blob existence check!")

✅ Migration functions updated to use blob existence check!


## 4. List Existing Table Storage Conversations

Query and inspect all conversation entities currently in Table Storage to understand what we're migrating.

In [49]:
# Query all conversations from Table Storage
print("🔍 Querying all conversations from Table Storage...")
all_entities = list(old_chatlogs_table.list_entities())

print(f"📊 Found {len(all_entities)} total conversation entities")

# Analyze the conversations
conversation_analysis = []
blob_conversations = 0
chunked_conversations = 0
total_size = 0

for entity in all_entities:
    info = inspect_conversation_entity(entity)
    conversation_analysis.append(info)
    
    # Check if blob exists for this conversation using retrieve_conversation_blob
    blob_result = blob_store.retrieve_conversation_blob(info["username"], info["conversation_id"])
    if blob_result is not None:
        blob_conversations += 1
        info["already_migrated"] = True
    else:
        chunked_conversations += 1
        info["already_migrated"] = False
    
    total_size += info["total_size"]

# Create summary DataFrame
df_analysis = pd.DataFrame(conversation_analysis)

print(f"\n📈 Migration Status Summary:")
print(f"   💾 Already in Blob Storage: {blob_conversations}")
print(f"   🔗 Still Chunked in Table: {chunked_conversations}")
print(f"   📏 Total Data Size: {total_size:,} bytes ({total_size/1024/1024:.2f} MB)")

# Display sample of conversations to migrate
print(f"\n🎯 Conversations to Migrate (not yet in blob storage):")
chunked_df = df_analysis[df_analysis['already_migrated'] == False].copy()
if len(chunked_df) > 0:
    print(f"   Found {len(chunked_df)} conversations to migrate")
    
    # Show top 10 largest conversations
    chunked_df_sorted = chunked_df.sort_values('total_size', ascending=False)
    display_cols = ['username', 'conversation_id', 'title', 'chunk_count', 'total_size', 'last_updated']
    print("\n📋 Top 10 Largest Conversations to Migrate:")
    print(chunked_df_sorted[display_cols].head(10).to_string(index=False))
else:
    print("   ✅ All conversations already migrated to blob storage!")

🔍 Querying all conversations from Table Storage...
📊 Found 93 total conversation entities
✗ Failed to retrieve conversation blob Angus Weich/006cbb96-b46c-4a63-a268-93c1fd011beb.json: The specified blob does not exist.
RequestId:13fa6df0-501e-0055-4897-ec7dfd000000
Time:2025-07-04T03:52:56.0756711Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:13fa6df0-501e-0055-4897-ec7dfd000000
Time:2025-07-04T03:52:56.0756711Z</Message></Error>
✗ Failed to retrieve conversation blob Angus Weich/2b247207-9d23-494e-802c-e2140d7fb83a.json: The specified blob does not exist.
RequestId:13fa6e6e-501e-0055-4097-ec7dfd000000
Time:2025-07-04T03:52:56.1724524Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:13fa6e6e-501e-0055-4097-ec7dfd000000
Time:2025-07-04T03:52:56.1724524Z</Message></Error>


### 4.1 Inspect Individual Conversations (Optional)

Let's examine a specific conversation to understand the chunking structure.

In [54]:
import traceback
# Inspect a specific conversation to understand chunking structure
def inspect_specific_conversation(username: str, conversation_id: str):
    """Detailed inspection of a single conversation"""
    try:
        entity = old_chatlogs_table.get_entity(partition_key=username, row_key=conversation_id)
        print(f"🔍 Inspecting conversation: {conversation_id}")
        print(f"   Username: {username}")
        print(f"   Title: {entity.get('conversation_title', 'Untitled')}")
        print(f"   Last Updated: {entity.get('last_updated', 'Unknown')}")
        
        # Check if blob exists for this conversation using retrieve_conversation_blob
        already_migrated = blob_store.retrieve_conversation_blob(username, conversation_id) is not None
        print(f"   Already Migrated to Blob: {already_migrated}")
        
        # Show chunk structure
        message_keys = [key for key in entity.keys() if key.startswith("messages")]
        print(f"   📦 Message chunks: {len(message_keys)}")
        
        for i, key in enumerate(sorted(message_keys)):
            chunk_size = len(entity[key]) if entity[key] else 0
            preview = entity[key][:100] + "..." if len(entity[key]) > 100 else entity[key]
            print(f"      {key}: {chunk_size:,} bytes")
            print(f"         Preview: {preview}")
            if i >= 2:  # Limit preview to first 3 chunks
                print(f"         ... and {len(message_keys) - 3} more chunks")
                break
        
        # Test JSON reconstruction
        print(f"\n🧪 Testing JSON reconstruction...")
        messages, error = reconstruct_conversation_json(entity)
        if error:
            print(f"   ❌ Reconstruction failed: {error}")
            return False
        else:
            print(f"   ✅ Successfully reconstructed {len(messages)} messages")
            print(f"   📝 Sample message types: {[msg.get('role', 'unknown') for msg in messages[:3]]}")
            return True
            
    except Exception as e:
        print(f"❌ Failed to inspect conversation: {e}")
        traceback.print_exc()
        return False

# Select a conversation to inspect
if len(chunked_df) > 0:
    # Pick the largest not-yet-migrated conversation for inspection
    sample_conversation = chunked_df_sorted.iloc[0]
    print(f"📋 Inspecting largest not-yet-migrated conversation:")
    success = inspect_specific_conversation(
        sample_conversation['username'], 
        sample_conversation['conversation_id']
    )
else:
    print("ℹ️ No conversations available for inspection")

📋 Inspecting largest not-yet-migrated conversation:
🔍 Inspecting conversation: 10bdac8e-a16e-4fe7-8192-a7567c422633
   Username: Martin Sawyers
   Title: Discussing Fatigue in Safety Reports
   Last Updated: 2024-12-19 03:25:29
✗ Failed to retrieve conversation blob Martin Sawyers/10bdac8e-a16e-4fe7-8192-a7567c422633.json: The specified blob does not exist.
RequestId:13fc64bc-501e-0055-6697-ec7dfd000000
Time:2025-07-04T03:55:51.3241027Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:13fc64bc-501e-0055-6697-ec7dfd000000
Time:2025-07-04T03:55:51.3241027Z</Message></Error>
   Already Migrated to Blob: False
   📦 Message chunks: 15
      messages_0: 30,000 bytes
         Preview: [{"role": "user", "metadata": {"title": null}, "content": "What has TAIC, ATSB and TSB said about fa...
      messages_1: 30,000 bytes
         Preview:  <td>2</td>\n      <td>TSB</td>\n      <td>Contact</t

### 4.2 JSON Validation and Concatenation Testing

Before migrating, let's ensure that all chunked conversations can be properly concatenated and parsed as valid JSON.

In [55]:
def validate_all_chunked_conversations():
    """
    Validate that all not-yet-migrated conversations can be properly reconstructed as JSON.
    This is crucial before migration to identify any data corruption issues.
    """
    print("🧪 Validating JSON reconstruction for all not-yet-migrated conversations...")
    
    validation_results = []
    valid_count = 0
    invalid_count = 0
    
    # Filter to conversations not yet migrated to blob using retrieve_conversation_blob
    chunked_entities = [entity for entity in all_entities if blob_store.retrieve_conversation_blob(entity.get("PartitionKey"), entity.get("RowKey")) is None]
    
    print(f"📊 Testing {len(chunked_entities)} not-yet-migrated conversations...")
    
    for i, entity in enumerate(chunked_entities):
        conversation_id = entity.get("RowKey", "unknown")
        username = entity.get("PartitionKey", "unknown")
        
        # Test reconstruction
        messages, error = reconstruct_conversation_json(entity)
        
        result = {
            "username": username,
            "conversation_id": conversation_id,
            "title": entity.get("conversation_title", "Untitled")[:50],
            "chunk_count": len([k for k in entity.keys() if k.startswith("messages")]),
            "is_valid": error is None,
            "error": error,
            "message_count": len(messages) if messages else 0
        }
        
        validation_results.append(result)
        
        if error is None:
            valid_count += 1
            if i < 5:  # Show details for first few
                print(f"   ✅ {conversation_id}: {len(messages)} messages, {result['chunk_count']} chunks")
        else:
            invalid_count += 1
            print(f"   ❌ {conversation_id}: {error}")
        
        # Progress indicator
        if (i + 1) % 10 == 0:
            print(f"   📈 Progress: {i + 1}/{len(chunked_entities)} tested...")
    
    # Summary
    print(f"\n📊 Validation Summary:")
    print(f"   ✅ Valid conversations: {valid_count}")
    print(f"   ❌ Invalid conversations: {invalid_count}")
    print(f"   📈 Success rate: {valid_count/(valid_count+invalid_count)*100:.1f}%")
    
    # Show invalid conversations with detailed information
    invalid_results = [r for r in validation_results if not r["is_valid"]]
    if invalid_results:
        print(f"\n⚠️ Invalid Conversations (will be skipped in migration):")
        for result in invalid_results:
            print(f"   🚫 User: {result['username']}")
            print(f"      Title: {result['title']}")
            print(f"      ID: {result['conversation_id']}")
            print(f"      Error: {result['error']}")
            print(f"      Chunks: {result['chunk_count']}")
            print()
    
    return validation_results

# Run validation
validation_results = validate_all_chunked_conversations()

🧪 Validating JSON reconstruction for all not-yet-migrated conversations...
✗ Failed to retrieve conversation blob Angus Weich/006cbb96-b46c-4a63-a268-93c1fd011beb.json: The specified blob does not exist.
RequestId:13fc7098-501e-0055-7697-ec7dfd000000
Time:2025-07-04T03:55:55.8209280Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:13fc7098-501e-0055-7697-ec7dfd000000
Time:2025-07-04T03:55:55.8209280Z</Message></Error>
✗ Failed to retrieve conversation blob Angus Weich/2b247207-9d23-494e-802c-e2140d7fb83a.json: The specified blob does not exist.
RequestId:13fc70c7-501e-0055-2197-ec7dfd000000
Time:2025-07-04T03:55:55.9123765Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:13fc70c7-501e-0055-2197-ec7dfd000000
Time:2025-07-04T03:55:55.9123765Z</Message></Error>
✗ Failed to ret

## 5. Execute Migration

Now let's perform the actual migration of valid conversations from Table Storage chunks to Blob Storage.

In [56]:
def execute_migration(dry_run=True):
    """
    Execute the migration process.
    
    Args:
        dry_run: If True, only simulate migration without making changes
    """
    print(f"🚀 Starting migration (dry_run={dry_run})...")
    
    # Filter to valid not-yet-migrated conversations using retrieve_conversation_blob
    valid_chunked = [
        entity for entity in all_entities 
        if blob_store.retrieve_conversation_blob(entity.get("PartitionKey"), entity.get("RowKey")) is None
    ]
    
    migration_results = []
    success_count = 0
    skip_count = 0
    fail_count = 0
    
    print(f"📊 Migrating {len(valid_chunked)} conversations...")
    
    for i, entity in enumerate(valid_chunked):
        conversation_id = entity.get("RowKey", "unknown")
        username = entity.get("PartitionKey", "unknown")
        
        # Check if this conversation was validated as valid
        validation_result = next(
            (r for r in validation_results 
             if r["conversation_id"] == conversation_id and r["username"] == username), 
            None
        )
        
        if validation_result and not validation_result["is_valid"]:
            result = {
                "username": username,
                "conversation_id": conversation_id,
                "status": "skipped",
                "error": "Failed validation",
                "message_count": 0
            }
            skip_count += 1
            print(f"   ⏭️ Skipped {conversation_id}: Failed validation")
        else:
            if dry_run:
                # Simulate migration
                result = {
                    "username": username,
                    "conversation_id": conversation_id,
                    "status": "simulated",
                    "error": None,
                    "message_count": validation_result["message_count"] if validation_result else 0
                }
                success_count += 1
                if i < 5:  # Show details for first few
                    print(f"   🔄 Would migrate {conversation_id}: {result['message_count']} messages")
            else:
                # Actual migration
                result = migrate_single_conversation(entity)
                if result["status"] == "success":
                    success_count += 1
                    print(f"   ✅ Migrated {conversation_id}: {result['message_count']} messages")
                elif result["status"] == "already_migrated":
                    skip_count += 1
                    print(f"   ⏭️ Already migrated {conversation_id}")
                else:
                    fail_count += 1
                    print(f"   ❌ Failed {conversation_id}: {result['error']}")
        
        migration_results.append(result)
        
        # Progress indicator
        if (i + 1) % 10 == 0:
            print(f"   📈 Progress: {i + 1}/{len(valid_chunked)} processed...")
    
    # Summary
    print(f"\n📊 Migration Summary:")
    print(f"   ✅ Successful: {success_count}")
    print(f"   ⏭️ Skipped: {skip_count}")
    print(f"   ❌ Failed: {fail_count}")
    print(f"   📈 Success rate: {success_count/(len(valid_chunked))*100:.1f}%")
    
    return migration_results

# First, run a dry-run to see what would happen
print("🎯 Running DRY RUN (no actual changes):")
dry_run_results = execute_migration(dry_run=True)

🎯 Running DRY RUN (no actual changes):
🚀 Starting migration (dry_run=True)...
✗ Failed to retrieve conversation blob Angus Weich/006cbb96-b46c-4a63-a268-93c1fd011beb.json: The specified blob does not exist.
RequestId:13fcaa52-501e-0055-2e97-ec7dfd000000
Time:2025-07-04T03:56:17.4450168Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:13fcaa52-501e-0055-2e97-ec7dfd000000
Time:2025-07-04T03:56:17.4450168Z</Message></Error>
✗ Failed to retrieve conversation blob Angus Weich/2b247207-9d23-494e-802c-e2140d7fb83a.json: The specified blob does not exist.
RequestId:13fcaaa6-501e-0055-5697-ec7dfd000000
Time:2025-07-04T03:56:17.5243738Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:13fcaaa6-501e-0055-5697-ec7dfd000000
Time:2025-07-04T03:56:17.5243738Z</Message></Error>
✗ Failed to 

In [9]:
# CAUTION: This will perform the actual migration!
# Uncomment the lines below only when you're ready to migrate

print("⚠️ ACTUAL MIGRATION - Uncomment the lines below to execute:")
print("# actual_results = execute_migration(dry_run=False)")

# Uncomment these lines when ready:
print("🚀 Running ACTUAL MIGRATION:")
actual_results = execute_migration(dry_run=False)

⚠️ ACTUAL MIGRATION - Uncomment the lines below to execute:
# actual_results = execute_migration(dry_run=False)
🚀 Running ACTUAL MIGRATION:
🚀 Starting migration (dry_run=False)...
📊 Migrating 93 conversations...
✓ Stored conversation blob: Angus Weich/006cbb96-b46c-4a63-a268-93c1fd011beb.json (117035 bytes)
✓ Stored conversation blob: Angus Weich/006cbb96-b46c-4a63-a268-93c1fd011beb.json (117035 bytes)
✓ Stored conversation metadata for 006cbb96-b46c-4a63-a268-93c1fd011beb
   ✅ Migrated 006cbb96-b46c-4a63-a268-93c1fd011beb: 8 messages
✓ Stored conversation metadata for 006cbb96-b46c-4a63-a268-93c1fd011beb
   ✅ Migrated 006cbb96-b46c-4a63-a268-93c1fd011beb: 8 messages
✓ Stored conversation blob: Angus Weich/2b247207-9d23-494e-802c-e2140d7fb83a.json (91844 bytes)
✓ Stored conversation blob: Angus Weich/2b247207-9d23-494e-802c-e2140d7fb83a.json (91844 bytes)
✓ Stored conversation metadata for 2b247207-9d23-494e-802c-e2140d7fb83a
   ✅ Migrated 2b247207-9d23-494e-802c-e2140d7fb83a: 4 messag

## 6. Verify Migration Results

After migration, let's verify that conversations can be properly retrieved from Blob Storage and that the JSON is intact.

In [ ]:
def verify_migration():
    """
    Verify that migrated conversations can be properly retrieved from Blob Storage.
    """
    print("🔍 Verifying migration results...")
    
    # Re-query all entities to get updated state
    updated_entities = list(chatlogs_table.list_entities())
    
    blob_count = 0
    chunked_count = 0
    verification_results = []
    
    for entity in updated_entities:
        conversation_id = entity.get("RowKey")
        username = entity.get("PartitionKey")
        
        if blob_store.retrieve_conversation_blob(username, conversation_id):
            blob_count += 1
            
            # Test retrieval from blob storage
            try:
                messages = blob_store.retrieve_conversation_blob(username, conversation_id)
                if messages is not None:
                    verification_results.append({
                        "conversation_id": conversation_id,
                        "username": username,
                        "status": "success",
                        "message_count": len(messages),
                        "error": None
                    })
                else:
                    verification_results.append({
                        "conversation_id": conversation_id,
                        "username": username,
                        "status": "failed",
                        "message_count": 0,
                        "error": "Could not retrieve blob"
                    })
            except Exception as e:
                verification_results.append({
                    "conversation_id": conversation_id,
                    "username": username,
                    "status": "error",
                    "message_count": 0,
                    "error": str(e)
                })
        else:
            chunked_count += 1
    
    print(f"\n📊 Migration Status After Verification:")
    print(f"   💾 Conversations in Blob Storage: {blob_count}")
    print(f"   🔗 Conversations still chunked: {chunked_count}")
    
    # Show verification results
    success_verifications = [r for r in verification_results if r["status"] == "success"]
    failed_verifications = [r for r in verification_results if r["status"] != "success"]
    
    print(f"\n✅ Blob Storage Verification:")
    print(f"   Successful retrievals: {len(success_verifications)}")
    print(f"   Failed retrievals: {len(failed_verifications)}")
    
    if len(success_verifications) > 0:
        total_messages = sum(r["message_count"] for r in success_verifications)
        print(f"   Total messages verified: {total_messages:,}")
    
    if failed_verifications:
        print(f"\n❌ Failed Verifications:")
        for result in failed_verifications[:5]:  # Show first 5 failures
            print(f"   {result['conversation_id']}: {result['error']}")
        if len(failed_verifications) > 5:
            print(f"   ... and {len(failed_verifications) - 5} more failures")
    
    return verification_results

# Run verification
verification_results = verify_migration()

In [ ]:
def test_conversation_retrieval_system():
    """
    Test the complete conversation retrieval system as it would work in the app.
    """
    print("🧪 Testing complete conversation retrieval system...")
    
    # Get all unique usernames
    usernames = list(set(entity.get("PartitionKey") for entity in all_entities))
    print(f"👥 Found {len(usernames)} unique users")
    
    for username in usernames[:3]:  # Test first 3 users
        print(f"\n👤 Testing user: {username}")
        
        try:
            # Use the same method as the app
            conversations = metadata_store.get_user_conversations(username)
            
            print(f"   📋 Retrieved {len(conversations)} conversations")
            
            # Test a few conversations
            for i, conv in enumerate(conversations[:2]):  # Test first 2 conversations
                print(f"   📝 Conversation {i+1}:")
                print(f"      ID: {conv['id']}")
                print(f"      Title: {conv['conversation_title']}")
                print(f"      Messages: {len(conv['messages'])}")
                print(f"      Storage: {conv.get('storage_type', 'unknown')}")
                print(f"      Last Updated: {conv['last_updated']}")
                
                # Verify JSON structure
                messages = conv['messages']
                if isinstance(messages, list) and len(messages) > 0:
                    first_msg = messages[0]
                    if isinstance(first_msg, dict) and 'role' in first_msg:
                        print(f"      ✅ Valid JSON structure")
                    else:
                        print(f"      ❌ Invalid message structure")
                else:
                    print(f"      ⚠️ Empty or invalid messages list")
                    
        except Exception as e:
            print(f"   ❌ Failed to retrieve conversations: {e}")
    
    print(f"\n✅ Conversation retrieval system test completed")

# Test the retrieval system
test_conversation_retrieval_system()

## 7. Cleanup (Optional)

After successful migration and verification, you may optionally clean up the old chunked data from Table Storage to save space.

In [ ]:
def cleanup_chunked_data(dry_run=True):
    """
    Clean up old chunked message data from Table Storage entities.
    This removes the messages_* columns while preserving metadata.
    
    Args:
        dry_run: If True, only simulate cleanup without making changes
    """
    print(f"🧹 Cleaning up chunked data (dry_run={dry_run})...")
    
    # Find entities that have been migrated to blob storage (by blob existence)
    cleanup_candidates = []
    
    for entity in all_entities:
        username = entity.get("PartitionKey")
        conversation_id = entity.get("RowKey")
        if blob_store.blob_exists(username, conversation_id):
            # Check if it has old chunked data
            message_keys = [key for key in entity.keys() if key.startswith("messages")]
            if message_keys:
                cleanup_candidates.append({
                    "entity": entity,
                    "message_keys": message_keys,
                    "username": username,
                    "conversation_id": conversation_id
                })
    
    print(f"📊 Found {len(cleanup_candidates)} entities with old chunked data to clean")
    
    if not cleanup_candidates:
        print("✅ No cleanup needed - no old chunked data found")
        return
    
    # Calculate space savings
    total_chunk_size = 0
    for candidate in cleanup_candidates:
        for key in candidate["message_keys"]:
            chunk_size = len(candidate["entity"][key]) if candidate["entity"][key] else 0
            total_chunk_size += chunk_size
    
    print(f"💾 Potential space savings: {total_chunk_size:,} bytes ({total_chunk_size/1024/1024:.2f} MB)")
    
    if dry_run:
        print("🔍 DRY RUN - Would clean up the following:")
        for candidate in cleanup_candidates[:5]:  # Show first 5
            print(f"   {candidate['conversation_id']}: {len(candidate['message_keys'])} chunks")
        if len(cleanup_candidates) > 5:
            print(f"   ... and {len(cleanup_candidates) - 5} more entities")
    else:
        print("⚠️ ACTUAL CLEANUP - This will permanently remove chunked data!")
        print("   The blob storage data will remain intact.")
        
        cleaned_count = 0
        for candidate in cleanup_candidates:
            try:
                entity = candidate["entity"]
                
                # Create a new entity without the message chunks
                cleaned_entity = {
                    key: value for key, value in entity.items()
                    if not key.startswith("messages")
                }
                
                # Update the entity
                chatlogs_table.update_entity(entity=cleaned_entity)
                cleaned_count += 1
                
                if cleaned_count <= 5:  # Show progress for first few
                    print(f"   ✅ Cleaned {candidate['conversation_id']}")
                
            except Exception as e:
                print(f"   ❌ Failed to clean {candidate['conversation_id']}: {e}")
        
        print(f"✅ Cleanup completed: {cleaned_count}/{len(cleanup_candidates)} entities cleaned")

print("⚠️ CLEANUP - Uncomment the lines below to execute:")
print("# cleanup_chunked_data(dry_run=True)   # Dry run first")
print("# cleanup_chunked_data(dry_run=False)  # Actual cleanup")

# Uncomment these lines when ready:
# print("🔍 Running CLEANUP DRY RUN:")
# cleanup_chunked_data(dry_run=True)

## 8. Remove Specific Columns from Old Chatlogs Table

Remove unwanted columns from all entities in the old chatlogs table. This is useful for cleaning up deprecated or unnecessary data.

In [38]:
def remove_columns_from_table(columns_to_remove: List[str], dry_run=True):
    """
    Remove specific columns from all entities in the old_chatlogs_table.
    
    Args:
        columns_to_remove: List of column names to remove from all entities
        dry_run: If True, only simulate the removal without making changes
    """
    print(f"🗑️ Removing columns from old_chatlogs_table (dry_run={dry_run})...")
    print(f"📋 Columns to remove: {', '.join(columns_to_remove)}")
    
    # Get all entities from the old table
    print("🔍 Querying all entities from old_chatlogs_table...")
    all_old_entities = list(old_chatlogs_table.list_entities())
    print(f"📊 Found {len(all_old_entities)} entities in old_chatlogs_table")
    
    # Analyze which entities have the columns to remove
    entities_with_columns = []
    total_removals = 0
    
    for entity in all_old_entities:
        columns_found = []
        for col in columns_to_remove:
            if col in entity:
                columns_found.append(col)
                total_removals += 1
        
        if columns_found:
            entities_with_columns.append({
                "entity": entity,
                "columns_found": columns_found,
                "username": entity.get("PartitionKey", "unknown"),
                "conversation_id": entity.get("RowKey", "unknown")
            })
    
    print(f"\n📈 Analysis Results:")
    print(f"   🎯 Entities with target columns: {len(entities_with_columns)}")
    print(f"   🗑️ Total column removals needed: {total_removals}")
    
    if not entities_with_columns:
        print("✅ No columns to remove - target columns not found in any entities")
        return
    
    # Show sample of what will be removed
    print(f"\n🔍 Sample entities that will be modified:")
    for i, item in enumerate(entities_with_columns[:5]):  # Show first 5
        print(f"   Entity {i+1}: {item['conversation_id']}")
        print(f"      User: {item['username']}")
        print(f"      Columns to remove: {', '.join(item['columns_found'])}")
    
    if len(entities_with_columns) > 5:
        print(f"   ... and {len(entities_with_columns) - 5} more entities")
    
    if dry_run:
        print(f"\n🔍 DRY RUN - Would remove columns from {len(entities_with_columns)} entities")
        
        # Show detailed preview for first entity
        if entities_with_columns:
            sample_entity = entities_with_columns[0]["entity"]
            print(f"\n📋 Sample entity modification preview:")
            print(f"   Original columns: {list(sample_entity.keys())}")
            
            cleaned_columns = [key for key in sample_entity.keys() 
                             if key not in columns_to_remove]
            print(f"   After removal: {cleaned_columns}")
            
            removed_columns = [key for key in sample_entity.keys() 
                             if key in columns_to_remove]
            print(f"   Removed columns: {removed_columns}")
    else:
        print(f"\n⚠️ ACTUAL REMOVAL - Removing columns from {len(entities_with_columns)} entities...")
        
        from azure.data.tables import UpdateMode
        
        success_count = 0
        fail_count = 0
        
        for i, item in enumerate(entities_with_columns):
            try:
                entity = item["entity"]
                conversation_id = item["conversation_id"]
                
                # Create new entity without the target columns
                cleaned_entity = {
                    key: value for key, value in entity.items()
                    if key not in columns_to_remove
                }
                
                # Ensure PartitionKey and RowKey are always present
                if "PartitionKey" not in cleaned_entity:
                    cleaned_entity["PartitionKey"] = entity.get("PartitionKey")
                if "RowKey" not in cleaned_entity:
                    cleaned_entity["RowKey"] = entity.get("RowKey")
                
                # Update the entity (replace completely)
                old_chatlogs_table.update_entity(entity=cleaned_entity, mode=UpdateMode.REPLACE)
                success_count += 1
                
                if success_count <= 5:  # Show progress for first few
                    removed_cols = item["columns_found"]
                    print(f"   ✅ Updated {conversation_id}: removed {', '.join(removed_cols)}")
                
                # Progress indicator
                if (i + 1) % 10 == 0:
                    print(f"   📈 Progress: {i + 1}/{len(entities_with_columns)} processed...")
                
            except Exception as e:
                fail_count += 1
                print(f"   ❌ Failed to update {conversation_id}: {e}")
        
        print(f"\n📊 Column Removal Summary:")
        print(f"   ✅ Successfully updated: {success_count}")
        print(f"   ❌ Failed updates: {fail_count}")
        print(f"   📈 Success rate: {success_count/(success_count+fail_count)*100:.1f}%")

# Specify the columns you want to remove
COLUMNS_TO_REMOVE = [
    "blob_name",
    'created_at',
    'storage_type',
]

print("🎯 COLUMN REMOVAL CONFIGURATION:")
print("⚠️  IMPORTANT: Update the COLUMNS_TO_REMOVE list above with the actual column names you want to remove!")
print(f"📋 Currently configured to remove: {COLUMNS_TO_REMOVE}")
print()
print("💡 To see what columns exist in your entities, run this first:")
print("   sample_entity = list(old_chatlogs_table.list_entities())[0]")
print("   print('Available columns:', list(sample_entity.keys()))")

🎯 COLUMN REMOVAL CONFIGURATION:
⚠️  IMPORTANT: Update the COLUMNS_TO_REMOVE list above with the actual column names you want to remove!
📋 Currently configured to remove: ['blob_name', 'created_at', 'storage_type']

💡 To see what columns exist in your entities, run this first:
   sample_entity = list(old_chatlogs_table.list_entities())[0]
   print('Available columns:', list(sample_entity.keys()))


In [39]:
# First, let's inspect what columns exist in the old_chatlogs_table
print("🔍 Inspecting columns in old_chatlogs_table...")

# Get a sample entity to see available columns
sample_entities = list(old_chatlogs_table.list_entities())

if sample_entities:
    sample_entity = sample_entities[0]
    all_columns = list(sample_entity.keys())
    
    print(f"📊 Sample entity has {len(all_columns)} columns:")
    
    # Group columns by type for better visualization
    system_columns = [col for col in all_columns if col in ['PartitionKey', 'RowKey', 'Timestamp', 'etag']]
    message_columns = [col for col in all_columns if col.startswith('messages')]
    metadata_columns = [col for col in all_columns if col not in system_columns and not col.startswith('messages')]
    
    print(f"\n🔧 System columns ({len(system_columns)}):")
    for col in system_columns:
        print(f"   - {col}")
    
    print(f"\n💬 Message columns ({len(message_columns)}):")
    for col in message_columns[:10]:  # Show first 10 message columns
        print(f"   - {col}")
    if len(message_columns) > 10:
        print(f"   ... and {len(message_columns) - 10} more message columns")
    
    print(f"\n📋 Metadata columns ({len(metadata_columns)}):")
    for col in metadata_columns:
        print(f"   - {col}")
    
    print(f"\n📝 All columns: {all_columns}")
    
else:
    print("❌ No entities found in old_chatlogs_table")

🔍 Inspecting columns in old_chatlogs_table...
📊 Sample entity has 12 columns:

🔧 System columns (2):
   - PartitionKey
   - RowKey

💬 Message columns (4):
   - messages_0
   - messages_1
   - messages_2
   - messages_3

📋 Metadata columns (6):
   - blob_name
   - conversation_title
   - created_at
   - last_updated
   - message_count
   - storage_type

📝 All columns: ['PartitionKey', 'RowKey', 'blob_name', 'conversation_title', 'created_at', 'last_updated', 'message_count', 'messages_0', 'messages_1', 'messages_2', 'messages_3', 'storage_type']
📊 Sample entity has 12 columns:

🔧 System columns (2):
   - PartitionKey
   - RowKey

💬 Message columns (4):
   - messages_0
   - messages_1
   - messages_2
   - messages_3

📋 Metadata columns (6):
   - blob_name
   - conversation_title
   - created_at
   - last_updated
   - message_count
   - storage_type

📝 All columns: ['PartitionKey', 'RowKey', 'blob_name', 'conversation_title', 'created_at', 'last_updated', 'message_count', 'messages_0', 'm

In [40]:
# Now specify the exact columns you want to remove and execute the removal
# Update this list with the actual column names you want to remove:

print("⚠️ BEFORE RUNNING: Update COLUMNS_TO_REMOVE with the actual column names!")
print(f"Currently set to remove: {COLUMNS_TO_REMOVE}")
print()
print("🔍 First, run a DRY RUN to see what would be removed:")
print("# remove_columns_from_table(COLUMNS_TO_REMOVE, dry_run=True)")
print()
print("🚀 Then run the actual removal:")  
print("# remove_columns_from_table(COLUMNS_TO_REMOVE, dry_run=False)")

# Uncomment and run these when you're ready:
print("🎯 Running DRY RUN:")
remove_columns_from_table(COLUMNS_TO_REMOVE, dry_run=True)

⚠️ BEFORE RUNNING: Update COLUMNS_TO_REMOVE with the actual column names!
Currently set to remove: ['blob_name', 'created_at', 'storage_type']

🔍 First, run a DRY RUN to see what would be removed:
# remove_columns_from_table(COLUMNS_TO_REMOVE, dry_run=True)

🚀 Then run the actual removal:
# remove_columns_from_table(COLUMNS_TO_REMOVE, dry_run=False)
🎯 Running DRY RUN:
🗑️ Removing columns from old_chatlogs_table (dry_run=True)...
📋 Columns to remove: blob_name, created_at, storage_type
🔍 Querying all entities from old_chatlogs_table...
📊 Found 93 entities in old_chatlogs_table

📈 Analysis Results:
   🎯 Entities with target columns: 79
   🗑️ Total column removals needed: 237

🔍 Sample entities that will be modified:
   Entity 1: 006cbb96-b46c-4a63-a268-93c1fd011beb
      User: Angus Weich
      Columns to remove: blob_name, created_at, storage_type
   Entity 2: 2b247207-9d23-494e-802c-e2140d7fb83a
      User: Angus Weich
      Columns to remove: blob_name, created_at, storage_type
   Ent

In [41]:
# EXECUTE ACTUAL COLUMN REMOVAL
# ⚠️ CAUTION: This will permanently remove the specified columns!

print("⚠️ ACTUAL COLUMN REMOVAL - Uncomment when ready:")
print("# remove_columns_from_table(COLUMNS_TO_REMOVE, dry_run=False)")

# Uncomment this line when you're ready to actually remove the columns:
remove_columns_from_table(COLUMNS_TO_REMOVE, dry_run=False)

⚠️ ACTUAL COLUMN REMOVAL - Uncomment when ready:
# remove_columns_from_table(COLUMNS_TO_REMOVE, dry_run=False)
🗑️ Removing columns from old_chatlogs_table (dry_run=False)...
📋 Columns to remove: blob_name, created_at, storage_type
🔍 Querying all entities from old_chatlogs_table...
📊 Found 93 entities in old_chatlogs_table

📈 Analysis Results:
   🎯 Entities with target columns: 79
   🗑️ Total column removals needed: 237

🔍 Sample entities that will be modified:
   Entity 1: 006cbb96-b46c-4a63-a268-93c1fd011beb
      User: Angus Weich
      Columns to remove: blob_name, created_at, storage_type
   Entity 2: 2b247207-9d23-494e-802c-e2140d7fb83a
      User: Angus Weich
      Columns to remove: blob_name, created_at, storage_type
   Entity 3: 727bcfd7-8f9c-4825-9a8f-d4c5d3d577bf
      User: Angus Weich
      Columns to remove: blob_name, created_at, storage_type
   Entity 4: 7465795c-9ffd-4a8f-9b85-41a88eb9a720
      User: Angus Weich
      Columns to remove: blob_name, created_at, storage_